# **Assignment 2**

## Framing the Problem


Industrial robotic arms often require extremely high precision to complete their tasks accurately and efficiently. Any deviation from expected performance can lead to defects, increased downtime, or even damage to equipment. Therefore, implementing real-time anomaly detection is crucial for ensuring smooth and reliable operation.  

The primary business objective of real-time anomaly detection is to identify irregularities during operation before they escalate into critical failures. By continuously monitoring sensor data, the system can detect deviations from normal patterns. This allows for immediate corrective action, such as halting the process or triggering automated adjustments.  

By preventing potential failures early, businesses can reduce waste, minimize downtime, and maintain high-quality production standards. Additionally, proactive anomaly detection enhances equipment longevity, reduces maintenance costs, and improves overall operational efficiency



---
We will explore three different methods for anomaly detection: K-Means, DBSCAN and Gaussian Mixture.

Performance should be measured by validating the model on nomal data found in ``IMU_20Hz_freq_drop.csv`` and  `IMU_10Hz`. A suitable threshold will be chosen to detect unnormal behaviour over a time window.

* For K-Means we will compare the inertias of the clusters to the inertias generated from normal data.

* For DBSCAN we will use the fraction of datapoints that fall outside clusters.

* For Gaussian Mixture we will use the mean log-likelihood to compare with a threshold.



### Assumptions

* Due to how the IMUs are set up for data collection, we assume some variations in data quality. The IMUs are placed on a breadboard that is secured with a strap. This likely explains the minor deviations found in the normal data recorded from ``IMU_20Hz_freq_drop.csv``. To account for this, we will simply adjust the threshold accordingly.

* After testing with resampling techniques to maintain a consistent sampling frequency, without any significant differences, we assume that the sampling frequency of the different IMUs does not impose any difference.

# Collecting the Data

For anomaly detection to work, we need a sequential process  data of 

The data is collected from here:
https://www.kaggle.com/datasets/hkayan/industrial-robotic-arm-imu-data-casper-1-and-2?fbclid=IwY2xjawI3yJdleHRuA2FlbQIxMAABHeB7cl5kMDV8YOgyqVeK-oP1pkBnbs1WEpvr3ZeDjBVKGANccmx1C2IKeg_aem_ZMw13qbiYzDAL9aYGz2JHA